In [1]:
!curl localhost:9200

{
  "name" : "3177cfcf4526",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "-7ty4Dc6RP63v4i9eQm5bA",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [5]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [1]:
from elasticsearch import Elasticsearch

In [2]:
es_client = Elasticsearch('http://localhost:9200') 

In [3]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)


  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [00:04<00:00, 194.80it/s]


In [8]:
query = 'How do execute a command on a Kubernetes pod?'

In [11]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    return response
    
    # result_docs = []
    
    # for hit in response['hits']['hits']:
    #     result_docs.append(hit['_source'])
    
    # return result_docs

In [13]:
search_results = elastic_search(query)
for hit in search_results['hits']['hits']:
    print(f"Score: {hit['_score']}")
    print(f"Section: {hit['_source']['section']}")
    print(f"Question: {hit['_source']['question']}")
    print(f"Text: {hit['_source']['text'][:200]}...")  # Print first 200 characters
    print("-" * 80)

Score: 44.50556
Section: 5. Deploying Machine Learning Models
Question: How do I debug a docker container?
Text: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a ...
--------------------------------------------------------------------------------
Score: 35.433445
Section: 10. Kubernetes and TensorFlow Serving
Question: Kubernetes-dashboard
Text: Deploy and Access the Kubernetes Dashboard
Luke...
--------------------------------------------------------------------------------
Score: 33.70974
Section: 5. Deploying Machine Learning Models
Question: How do I copy files from a different folder into docker container’s working directory?
Text: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
In the Dockerfile, you can provide the folder containing the files that you want to...
